In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# --- Step 1: Hyperparameters and Setup ---
# Define key training parameters
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHS = 5

In [ ]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# --- Step 2: Load and Prepare the Data ---
# Define image transformations (convert to tensor)
transform = transforms.ToTensor()


In [ ]:
# Download and load the training data
train_dataset = datasets.FashionMNIST(
    root='./data', 
    train=True, 
    transform=transform, 
    download=True
)

In [ ]:
# Create a DataLoader for the training set
train_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

In [ ]:
# Download and load the testing data
test_dataset = datasets.FashionMNIST(
    root='./data', 
    train=False, 
    transform=transform, 
    download=True
)

In [ ]:
# Create a DataLoader for the testing set
test_loader = DataLoader(
    dataset=test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False
)

In [ ]:
# --- Step 3: Define the Convolutional Neural Network (CNN) ---
class CoolCNN(nn.Module):
    def __init__(self):
        super(CoolCNN, self).__init__()
        # First convolutional layer: 1 input channel (grayscale), 16 output channels
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        # Max-pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Second convolutional layer: 16 input channels, 32 output channels
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        # Fully connected layers
        # The input size is calculated based on the output of the conv/pool layers
        self.fc1 = nn.Linear(32 * 7 * 7, 128) 
        self.fc2 = nn.Linear(128, 10) # 10 classes for Fashion MNIST
    def forward(self, x):
        # -> n_samples, 1, 28, 28
        x = self.pool(torch.relu(self.conv1(x)))
        # -> n_samples, 16, 14, 14
        x = self.pool(torch.relu(self.conv2(x)))
        # -> n_samples, 32, 7, 7

        # Flatten the feature maps for the fully connected layers
        x = x.view(-1, 32 * 7 * 7)
        # -> n_samples, 32*7*7

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        # -> n_samples, 10
        return x


In [ ]:
# Instantiate the model and move it to the GPU/CPU
model = CoolCNN().to(device)

In [ ]:
# --- Step 4: Define Loss and Optimizer ---
# Cross-entropy loss for multi-class classification
criterion = nn.CrossEntropyLoss()

In [ ]:
# Adam optimizer is a good choice for most tasks
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [ ]:
# --- Step 5: Training Loop ---
print("Starting training...")
for epoch in range(NUM_EPOCHS):
    for images, labels in train_loader:
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {loss.item():.4f}')
print("Training finished.")

In [ ]:
# --- Step 6: Test the Model ---
# Set the model to evaluation mode
model.eval()  
with torch.no_grad():
    correct = 0
    total = 0